<a href="https://colab.research.google.com/github/IngaSamoneneko/Yoga_Classification/blob/master/Yoga_Classification-Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [214]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from math import ceil
from sklearn.metrics import f1_score, classification_report

In [215]:
# ==========================
# CONFIG
# ==========================
ROOT_PATH  = '/content/drive/MyDrive/ML_Bootcamp_Data'
IMAGE_PATH = os.path.join(ROOT_PATH, 'images')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
NUM_CLASSES = 6
EPOCHS = 15  # Set higher after testing
IMG_SIZE = 128  # Smaller for faster training

print("Використовується пристрій:", DEVICE)

Використовується пристрій: cuda


In [216]:
# ==========================
# ЗАВАНТАЖЕННЯ ДАНИХ
# ==========================
train_df = pd.read_csv(os.path.join(ROOT_PATH, "train.csv"))
train_df, val_df = train_test_split(
    train_df, test_size=0.2,
    stratify=train_df["class_6"], random_state=42
)

In [217]:
# ==========================
# ТРАНСФОРМИ
# ==========================
train_transform = transforms.Compose([
    #transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomResizedCrop((IMG_SIZE, IMG_SIZE), scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [218]:
# ==========================
# КЛАС Dataset
# ==========================
class YogaDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, folder="train_images", has_labels=True):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.folder = folder
        self.has_labels = has_labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["image_id"]
        img_path = os.path.join(self.root_dir, self.folder, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.has_labels:
            label = self.df.iloc[idx]["class_6"]
            return image, label
        else:
            return image

In [219]:
# ==========================
# DATA LOADERS
# ==========================
train_dataset = YogaDataset(df=train_df, root_dir=IMAGE_PATH, transform=train_transform, folder="train_images", has_labels=True)
val_dataset = YogaDataset(df=val_df, root_dir=IMAGE_PATH, transform=val_transform, folder="train_images", has_labels=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1,pin_memory=True)

In [220]:
# ==========================
# ПРОСТА СNN-МОДЕЛЬ
# ==========================
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64, NUM_CLASSES)
        )

    def forward(self, x):
        return self.net(x)

model = SimpleCNN().to(DEVICE)

In [228]:
print(f"Пристрій: {DEVICE} | Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")
for epoch in range(1, EPOCHS + 1):
    train_one_epoch(model, train_loader, criterion, optimizer, epoch)
    val_acc = evaluate(model, val_loader)
    print(f"[Епоха {epoch}] Val Acc: {val_acc:.2f}%")
    scheduler.step(val_acc)

Пристрій: cuda | Train samples: 1888 | Val samples: 472
[Епоха 1] Train Loss: 1.7934 | Train Acc: 16.05%
[Епоха 1] Val Acc: 17.58%
[Епоха 2] Train Loss: 1.7931 | Train Acc: 16.15%
[Епоха 2] Val Acc: 17.58%
[Епоха 3] Train Loss: 1.7933 | Train Acc: 16.31%
[Епоха 3] Val Acc: 17.58%
[Епоха 4] Train Loss: 1.7930 | Train Acc: 16.58%
[Епоха 4] Val Acc: 17.58%


KeyboardInterrupt: 

In [208]:
# ==========================
# TRANSFER LEARNING: ResNet18 (оновлений синтаксис)
# ==========================
# Завантажуємо ResNet18 з рекомендованими ImageNet-вагами
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)

# Розморожуємо layer2, layer3 і layer4
for name, param in resnet.named_parameters():
    if name.startswith("layer2") or name.startswith("layer3") or name.startswith("layer4"):
        param.requires_grad = True

# Заміна останнього fc-шару: вхід = num_ftrs, вихід = 6 класів
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, NUM_CLASSES)
)

model = resnet.to(DEVICE)


In [221]:
# ==========================
# CLASS WEIGHTS
# ==========================
class_weights = compute_class_weight('balanced', classes=np.unique(train_df['class_6']), y=train_df['class_6'])
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

In [222]:
# ==========================
# LOSS, OPTIMIZER
# ==========================
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
#optimizer = optim.Adam(model.parameters(), lr=0.001)
params_to_update = [
    {"params": resnet.layer2.parameters(), "lr": 1e-5},
    {"params": resnet.layer3.parameters(), "lr": 1e-4},
    {"params": resnet.layer4.parameters(), "lr": 1e-4},
    {"params": resnet.fc.parameters(),    "lr": 1e-3},
]

optimizer = optim.Adam(params_to_update, weight_decay=1e-4)


scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',         # спираємося на максимізацію Validation Accuracy
    factor=0.5,         # зменшувати LR вполовину
    patience=2
)

#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)


In [227]:
# ==========================
# ФУНКЦІЯ ВАЛІДАЦІЇ
# ==========================
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total


def evaluate_with_f1(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    # Обчислюємо F1-метрику (зважене середнє по класах)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    # Додатково можна вивести classification report
    report = classification_report(all_labels, all_preds, digits=4)
    return f1, report

In [224]:
# ==========================
# ФУНКЦІЯ ТРЕНУВАННЯ ЗА ЕПОХУ
# ==========================
def train_one_epoch(model, loader, criterion, optimizer, epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    avg_loss = running_loss / len(loader)
    train_acc = 100 * correct / total
    print(f"[Епоха {epoch}] Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}%")



In [200]:
# ==========================
# ТРЕНУВАННЯ ТА ВАЛІДАЦІЯ
# ==========================
print(f"Пристрій: {DEVICE} | Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")
for epoch in range(1, EPOCHS + 1):
    train_one_epoch(model, train_loader, criterion, optimizer, epoch)
    val_acc = evaluate(model, val_loader)
    print(f"[Епоха {epoch}] Val Acc: {val_acc:.2f}%")
    scheduler.step(val_acc)

Пристрій: cuda | Train samples: 1888 | Val samples: 472
[Епоха 1] Train Loss: 1.7488 | Train Acc: 30.40%
[Епоха 1] Val Acc: 48.94%
[Епоха 2] Train Loss: 1.2070 | Train Acc: 53.71%
[Епоха 2] Val Acc: 56.78%
[Епоха 3] Train Loss: 0.9737 | Train Acc: 62.87%
[Епоха 3] Val Acc: 57.84%
[Епоха 4] Train Loss: 0.7852 | Train Acc: 69.33%
[Епоха 4] Val Acc: 63.98%
[Епоха 5] Train Loss: 0.6877 | Train Acc: 72.67%
[Епоха 5] Val Acc: 65.04%
[Епоха 6] Train Loss: 0.5291 | Train Acc: 79.18%
[Епоха 6] Val Acc: 67.37%
[Епоха 7] Train Loss: 0.4302 | Train Acc: 83.21%
[Епоха 7] Val Acc: 68.43%
[Епоха 8] Train Loss: 0.3692 | Train Acc: 86.28%
[Епоха 8] Val Acc: 70.55%
[Епоха 9] Train Loss: 0.3370 | Train Acc: 87.76%
[Епоха 9] Val Acc: 68.22%
[Епоха 10] Train Loss: 0.2771 | Train Acc: 89.14%
[Епоха 10] Val Acc: 71.40%
[Епоха 11] Train Loss: 0.2426 | Train Acc: 91.21%
[Епоха 11] Val Acc: 72.25%
[Епоха 12] Train Loss: 0.2158 | Train Acc: 92.32%
[Епоха 12] Val Acc: 73.31%
[Епоха 13] Train Loss: 0.1835 | Train 

In [213]:
best_val_f1 = 0.0

for epoch in range(1, EPOCHS + 1):
    train_one_epoch(model, train_loader, criterion, optimizer, epoch)
    val_acc = evaluate(model, val_loader)
    val_f1, val_report = evaluate_with_f1(model, val_loader)
    print(f"[Епоха {epoch}] Val Acc: {val_acc:.2f}%  |  Val F1 (weighted): {val_f1:.4f}")
    print("Classification Report:\n", val_report)
    scheduler.step(val_acc)

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1

print(f"Найкращий Val F1 (weighted): {best_val_f1:.4f}")

[Епоха 1] Train Loss: 1.8077 | Train Acc: 28.92%
[Епоха 1] Val Acc: 46.19%  |  Val F1 (weighted): 0.4479
Classification Report:
               precision    recall  f1-score   support

           0     0.4643    0.7723    0.5799       101
           1     0.4375    0.5000    0.4667        98
           2     0.4286    0.4355    0.4320        62
           3     0.4375    0.3684    0.4000        38
           4     0.6667    0.2975    0.4114       121
           5     0.3256    0.2692    0.2947        52

    accuracy                         0.4619       472
   macro avg     0.4600    0.4405    0.4308       472
weighted avg     0.4885    0.4619    0.4479       472

[Епоха 2] Train Loss: 1.2815 | Train Acc: 51.54%
[Епоха 2] Val Acc: 49.79%  |  Val F1 (weighted): 0.4812
Classification Report:
               precision    recall  f1-score   support

           0     0.5338    0.7030    0.6068       101
           1     0.4677    0.5918    0.5225        98
           2     0.4078    0.6774   